# Experiment 066: Add Fragprints to Best Model

Adding Fragprints features to our best GP + MLP + LGBM ensemble.
Using MSE metric (not MAE).
Best baseline: CV 0.008194 (exp_032)

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

DATA_PATH = '/home/data'

Using device: cuda


In [2]:
# Load data
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)
FRAGPRINTS_DF = pd.read_csv(f'{DATA_PATH}/fragprints_lookup.csv', index_col=0)

# Filter to non-zero variance
drfp_variance = DRFP_DF.var()
DRFP_FILTERED = DRFP_DF[drfp_variance[drfp_variance > 0].index.tolist()]

frag_variance = FRAGPRINTS_DF.var()
FRAGPRINTS_FILTERED = FRAGPRINTS_DF[frag_variance[frag_variance > 0].index.tolist()]

print(f'Spange: {SPANGE_DF.shape}')
print(f'DRFP filtered: {DRFP_FILTERED.shape}')
print(f'ACS PCA: {ACS_PCA_DF.shape}')
print(f'Fragprints filtered: {FRAGPRINTS_FILTERED.shape}')

Spange: (26, 13)
DRFP filtered: (24, 122)
ACS PCA: (24, 5)
Fragprints filtered: (24, 144)


In [3]:
# Data loading
INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = (X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"])
        yield (X[~mask], Y[~mask]), (X[mask], Y[mask])

print('Data loading functions defined')

Data loading functions defined


In [4]:
# Featurizer with optional Fragprints
class FullFeaturizer:
    def __init__(self, mixed=False, include_fragprints=False):
        self.mixed = mixed
        self.include_fragprints = include_fragprints
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.frag_df = FRAGPRINTS_FILTERED
        
        base_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]
        if include_fragprints:
            self.feats_dim = base_dim + self.frag_df.shape[1]
        else:
            self.feats_dim = base_dim

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1) / 100.0
            
            if flip:
                X_spange = B_spange * (1-pct) + A_spange * pct
                X_drfp = B_drfp * (1-pct) + A_drfp * pct
                X_acs = B_acs * (1-pct) + A_acs * pct
            else:
                X_spange = A_spange * (1-pct) + B_spange * pct
                X_drfp = A_drfp * (1-pct) + B_drfp * pct
                X_acs = A_acs * (1-pct) + B_acs * pct
            
            if self.include_fragprints:
                A_frag = self.frag_df.loc[X["SOLVENT A NAME"]].values
                B_frag = self.frag_df.loc[X["SOLVENT B NAME"]].values
                if flip:
                    X_frag = B_frag * (1-pct) + A_frag * pct
                else:
                    X_frag = A_frag * (1-pct) + B_frag * pct
                return np.hstack([X_kinetic, X_spange, X_drfp, X_acs, X_frag])
            else:
                return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
            
            if self.include_fragprints:
                X_frag = self.frag_df.loc[X["SOLVENT NAME"]].values
                return np.hstack([X_kinetic, X_spange, X_drfp, X_acs, X_frag])
            else:
                return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])
    
    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip), dtype=torch.double)

print('Featurizer defined')

Featurizer defined


In [5]:
# Simple Featurizer for GP (Spange + Arrhenius only)
class SimpleFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1]  # 18 features

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1) / 100.0
            if flip:
                X_spange = B_spange * (1-pct) + A_spange * pct
            else:
                X_spange = A_spange * (1-pct) + B_spange * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange])

print('Simple featurizer defined')

Simple featurizer defined


In [6]:
# GP Wrapper
class GPWrapper:
    def __init__(self, data='single'):
        self.data_type = data
        self.featurizer = SimpleFeaturizer(mixed=(data=='full'))
        self.models = []
        self.scaler = None

    def train_model(self, X_train, y_train):
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = y_train.values
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_std, X_flip])
            y_all = np.vstack([y_vals, y_vals])
        else:
            X_all, y_all = X_std, y_vals
        
        self.scaler = StandardScaler()
        X_scaled = self.scaler.fit_transform(X_all)
        
        self.models = []
        kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
        
        for i in range(3):
            gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True, random_state=42)
            gp.fit(X_scaled, y_all[:, i])
            self.models.append(gp)

    def predict(self, X_test):
        X_feat = self.featurizer.featurize(X_test, flip=False)
        X_scaled = self.scaler.transform(X_feat)
        preds = np.column_stack([m.predict(X_scaled) for m in self.models])
        return torch.tensor(preds, dtype=torch.double)

print('GP wrapper defined')

GP wrapper defined


In [7]:
# MLP Model
class MLPModelInternal(nn.Module):
    def __init__(self, input_dim, hidden_dims=[32, 16], output_dim=3, dropout=0.05):
        super().__init__()
        layers = [nn.BatchNorm1d(input_dim)]
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([nn.Linear(prev_dim, h_dim), nn.BatchNorm1d(h_dim), nn.ReLU(), nn.Dropout(dropout)])
            prev_dim = h_dim
        layers.extend([nn.Linear(prev_dim, output_dim), nn.Sigmoid()])
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

class WeightedMLPEnsemble:
    def __init__(self, hidden_dims=[32, 16], n_models=5, data='single', include_fragprints=False):
        self.hidden_dims = hidden_dims
        self.n_models = n_models
        self.data_type = data
        self.featurizer = FullFeaturizer(mixed=(data=='full'), include_fragprints=include_fragprints)
        self.models = []

    def train_model(self, X_train, y_train, epochs=200, batch_size=32, lr=5e-4):
        X_std = self.featurizer.featurize_torch(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize_torch(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all, y_all = X_std, y_vals
            
        input_dim = X_all.shape[1]
        self.models = []
        
        for i in range(self.n_models):
            torch.manual_seed(42 + i)
            model = MLPModelInternal(input_dim, self.hidden_dims).double().to(device)
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=20, factor=0.5)
            criterion = nn.HuberLoss()
            
            dataset = TensorDataset(X_all.to(device), y_all.to(device))
            loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            
            model.train()
            for epoch in range(epochs):
                epoch_loss = 0
                for X_batch, y_batch in loader:
                    optimizer.zero_grad()
                    pred = model(X_batch)
                    loss = criterion(pred, y_batch)
                    loss.backward()
                    optimizer.step()
                    epoch_loss += loss.item()
                scheduler.step(epoch_loss)
            
            self.models.append(model)

    def predict(self, X_test):
        X_feat = self.featurizer.featurize_torch(X_test, flip=False).to(device)
        preds = []
        for model in self.models:
            model.eval()
            with torch.no_grad():
                preds.append(model(X_feat).cpu())
        return torch.mean(torch.stack(preds), dim=0)

print('MLP ensemble defined')

MLP ensemble defined


In [8]:
# LGBM Wrapper
class LGBMWrapper:
    def __init__(self, data='single', include_fragprints=False):
        self.data_type = data
        self.featurizer = FullFeaturizer(mixed=(data=='full'), include_fragprints=include_fragprints)
        self.models = []

    def train_model(self, X_train, y_train):
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = y_train.values
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_std, X_flip])
            y_all = np.vstack([y_vals, y_vals])
        else:
            X_all, y_all = X_std, y_vals
        
        self.models = []
        params = {'objective': 'regression', 'metric': 'mse', 'boosting_type': 'gbdt',
                  'num_leaves': 31, 'learning_rate': 0.05, 'feature_fraction': 0.9,
                  'bagging_fraction': 0.8, 'bagging_freq': 5, 'verbose': -1, 'seed': 42}
        
        for i in range(3):
            train_data = lgb.Dataset(X_all, label=y_all[:, i])
            model = lgb.train(params, train_data, num_boost_round=100)
            self.models.append(model)

    def predict(self, X_test):
        X_feat = self.featurizer.featurize(X_test, flip=False)
        preds = np.column_stack([m.predict(X_feat) for m in self.models])
        return torch.tensor(preds, dtype=torch.double)

print('LGBM wrapper defined')

LGBM wrapper defined


In [9]:
# GP + MLP + LGBM Ensemble
class GPMLPLGBMEnsemble:
    def __init__(self, data='single', include_fragprints=False, gp_weight=0.15, mlp_weight=0.55, lgbm_weight=0.30):
        self.data_type = data
        self.include_fragprints = include_fragprints
        self.gp = GPWrapper(data=data)
        self.mlp = WeightedMLPEnsemble(hidden_dims=[32, 16], n_models=5, data=data, include_fragprints=include_fragprints)
        self.lgbm = LGBMWrapper(data=data, include_fragprints=include_fragprints)
        self.weights = {'gp': gp_weight, 'mlp': mlp_weight, 'lgbm': lgbm_weight}

    def train_model(self, X_train, y_train):
        self.gp.train_model(X_train, y_train)
        self.mlp.train_model(X_train, y_train)
        self.lgbm.train_model(X_train, y_train)

    def predict(self, X_test):
        gp_pred = self.gp.predict(X_test)
        mlp_pred = self.mlp.predict(X_test)
        lgbm_pred = self.lgbm.predict(X_test)
        
        combined = (self.weights['gp'] * gp_pred + 
                    self.weights['mlp'] * mlp_pred + 
                    self.weights['lgbm'] * lgbm_pred)
        return torch.clamp(combined, 0, 1)

print('Ensemble defined')

Ensemble defined


In [10]:
# Run CV without fragprints (baseline)
print('Running CV WITHOUT Fragprints (baseline)...')
print('='*60)

X, Y = load_data("single_solvent")
split_generator = generate_leave_one_out_splits(X, Y)

all_preds_base = []
all_actuals_base = []

for fold_idx, split in enumerate(split_generator):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = GPMLPLGBMEnsemble(data='single', include_fragprints=False)
    model.train_model(train_X, train_Y)
    
    predictions = model.predict(test_X)
    all_preds_base.append(predictions.numpy())
    all_actuals_base.append(test_Y.values)
    
    if fold_idx % 6 == 0:
        print(f'Fold {fold_idx+1}/24 complete')

all_preds_base = np.vstack(all_preds_base)
all_actuals_base = np.vstack(all_actuals_base)
mse_base = np.mean((all_preds_base - all_actuals_base) ** 2)
print(f'\nBaseline Single Solvent MSE: {mse_base:.6f}')

Running CV WITHOUT Fragprints (baseline)...


Fold 1/24 complete


Fold 7/24 complete


Fold 13/24 complete


Fold 19/24 complete



Baseline Single Solvent MSE: 0.008713


In [11]:
# Run CV WITH fragprints
print('\nRunning CV WITH Fragprints...')
print('='*60)

X, Y = load_data("single_solvent")
split_generator = generate_leave_one_out_splits(X, Y)

all_preds_frag = []
all_actuals_frag = []

for fold_idx, split in enumerate(split_generator):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = GPMLPLGBMEnsemble(data='single', include_fragprints=True)
    model.train_model(train_X, train_Y)
    
    predictions = model.predict(test_X)
    all_preds_frag.append(predictions.numpy())
    all_actuals_frag.append(test_Y.values)
    
    if fold_idx % 6 == 0:
        print(f'Fold {fold_idx+1}/24 complete')

all_preds_frag = np.vstack(all_preds_frag)
all_actuals_frag = np.vstack(all_actuals_frag)
mse_frag = np.mean((all_preds_frag - all_actuals_frag) ** 2)
print(f'\nWith Fragprints Single Solvent MSE: {mse_frag:.6f}')
print(f'Improvement: {(mse_base - mse_frag) / mse_base * 100:.2f}%')


Running CV WITH Fragprints...


Fold 1/24 complete


Fold 7/24 complete


Fold 13/24 complete


Fold 19/24 complete



With Fragprints Single Solvent MSE: 0.008217
Improvement: 5.69%


In [12]:
# Run full data CV with fragprints
print('\nRunning Full Data CV WITH Fragprints...')
print('='*60)

X_full, Y_full = load_data("full")
split_generator = generate_leave_one_ramp_out_splits(X_full, Y_full)

all_preds_full = []
all_actuals_full = []

for fold_idx, split in enumerate(split_generator):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = GPMLPLGBMEnsemble(data='full', include_fragprints=True)
    model.train_model(train_X, train_Y)
    
    predictions = model.predict(test_X)
    all_preds_full.append(predictions.numpy())
    all_actuals_full.append(test_Y.values)
    
    if fold_idx % 20 == 0:
        print(f'Fold {fold_idx+1} complete')

all_preds_full = np.vstack(all_preds_full)
all_actuals_full = np.vstack(all_actuals_full)
mse_full = np.mean((all_preds_full - all_actuals_full) ** 2)
print(f'\nFull Data MSE with Fragprints: {mse_full:.6f}')


Running Full Data CV WITH Fragprints...


Fold 1 complete



Full Data MSE with Fragprints: 0.019650


In [13]:
# Calculate combined CV score
n_single = len(all_preds_frag)
n_full = len(all_preds_full)

weighted_cv = (mse_frag * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n' + '='*60)
print(f'FINAL RESULTS')
print(f'='*60)
print(f'Single Solvent MSE (with Fragprints): {mse_frag:.6f} (n={n_single})')
print(f'Full Data MSE (with Fragprints): {mse_full:.6f} (n={n_full})')
print(f'Weighted Combined MSE: {weighted_cv:.6f}')
print(f'\nBest baseline CV: 0.008194')
print(f'Improvement: {(0.008194 - weighted_cv) / 0.008194 * 100:.2f}%')


FINAL RESULTS
Single Solvent MSE (with Fragprints): 0.008217 (n=656)
Full Data MSE (with Fragprints): 0.019650 (n=1227)
Weighted Combined MSE: 0.015667

Best baseline CV: 0.008194
Improvement: -91.20%


In [14]:
# Run full data CV WITHOUT fragprints (baseline)
print('\nRunning Full Data CV WITHOUT Fragprints (baseline)...')
print('='*60)

X_full, Y_full = load_data("full")
split_generator = generate_leave_one_ramp_out_splits(X_full, Y_full)

all_preds_full_base = []
all_actuals_full_base = []

for fold_idx, split in enumerate(split_generator):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = GPMLPLGBMEnsemble(data='full', include_fragprints=False)
    model.train_model(train_X, train_Y)
    
    predictions = model.predict(test_X)
    all_preds_full_base.append(predictions.numpy())
    all_actuals_full_base.append(test_Y.values)
    
    if fold_idx % 20 == 0:
        print(f'Fold {fold_idx+1} complete')

all_preds_full_base = np.vstack(all_preds_full_base)
all_actuals_full_base = np.vstack(all_actuals_full_base)
mse_full_base = np.mean((all_preds_full_base - all_actuals_full_base) ** 2)
print(f'\nFull Data MSE WITHOUT Fragprints: {mse_full_base:.6f}')


Running Full Data CV WITHOUT Fragprints (baseline)...


Fold 1 complete



Full Data MSE WITHOUT Fragprints: 0.019376


In [15]:
# Calculate combined CV for baseline (without Fragprints)
weighted_cv_base = (mse_base * n_single + mse_full_base * n_full) / (n_single + n_full)

print(f'\n' + '='*60)
print(f'COMPARISON')
print(f'='*60)
print(f'\nWITHOUT Fragprints:')
print(f'  Single Solvent MSE: {mse_base:.6f}')
print(f'  Full Data MSE: {mse_full_base:.6f}')
print(f'  Combined MSE: {weighted_cv_base:.6f}')

print(f'\nWITH Fragprints:')
print(f'  Single Solvent MSE: {mse_frag:.6f}')
print(f'  Full Data MSE: {mse_full:.6f}')
print(f'  Combined MSE: {weighted_cv:.6f}')

print(f'\nBest baseline CV: 0.008194')
print(f'Baseline (no frag) improvement: {(0.008194 - weighted_cv_base) / 0.008194 * 100:.2f}%')
print(f'With Fragprints improvement: {(0.008194 - weighted_cv) / 0.008194 * 100:.2f}%')


COMPARISON

WITHOUT Fragprints:
  Single Solvent MSE: 0.008713
  Full Data MSE: 0.019376
  Combined MSE: 0.015661

WITH Fragprints:
  Single Solvent MSE: 0.008217
  Full Data MSE: 0.019650
  Combined MSE: 0.015667

Best baseline CV: 0.008194
Baseline (no frag) improvement: -91.13%
With Fragprints improvement: -91.20%


In [16]:
# Debug: Check the number of ramps and samples per ramp
X_full, Y_full = load_data("full")
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
print(f"Number of ramps: {len(ramps)}")
print(f"Total samples: {len(X_full)}")

# Check samples per ramp
for idx, row in ramps.head(10).iterrows():
    mask = (X_full["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X_full["SOLVENT B NAME"] == row["SOLVENT B NAME"])
    print(f"Ramp: {row['SOLVENT A NAME'][:20]} + {row['SOLVENT B NAME'][:20]} = {mask.sum()} samples")

Number of ramps: 13
Total samples: 1227
Ramp: Methanol + Ethylene Glycol [1,2 = 122 samples
Ramp: 1,1,1,3,3,3-Hexafluo + 2-Methyltetrahydrofu = 124 samples
Ramp: Cyclohexane + IPA [Propan-2-ol] = 104 samples
Ramp: Water.Acetonitrile + Acetonitrile = 125 samples
Ramp: Acetonitrile + Acetonitrile.Acetic  = 125 samples
Ramp: 2-Methyltetrahydrofu + Diethyl Ether [Ether = 124 samples
Ramp: 2,2,2-Trifluoroethan + Water.2,2,2-Trifluor = 125 samples
Ramp: DMA [N,N-Dimethylace + Decanol = 110 samples
Ramp: Ethanol + THF [Tetrahydrofuran = 127 samples
Ramp: Dihydrolevoglucoseno + Ethyl Acetate = 36 samples


In [17]:
# Check RAMP NUM column
df_full = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
print(f"Unique RAMP NUM values: {df_full['RAMP NUM'].nunique()}")
print(f"RAMP NUM range: {df_full['RAMP NUM'].min()} to {df_full['RAMP NUM'].max()}")

Unique RAMP NUM values: 87
RAMP NUM range: 0 to 86
